In [56]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [57]:
data = pd.read_csv('data/apple.csv')
colume_drop = data.iloc[:, -2:]
data = data.drop(colume_drop, axis=1)
data['Date'] = pd.to_datetime(data['Date'], utc=True)
data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')
print(data.head())
print(type(data.index[0]))

                 Open       High        Low      Close     Volume
Date                                                             
2019-01-02  37.118910  38.067913  36.960742  37.845039  148158800
2019-01-03  34.504368  34.921355  34.029868  34.075401  365248800
2019-01-04  34.636170  35.599552  34.461229  35.530052  234428400
2019-01-07  35.635491  35.666646  34.964479  35.450962  219111200
2019-01-08  35.841593  36.383198  35.592362  36.126774  164101200
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


# 数据预处理

In [65]:
for column in data.columns:
    if data[column].isnull().sum() > 0:
        print(column)
    else:
        print(column, '无缺失值')
        
        

# 数据归一化

scaler = MinMaxScaler(feature_range=(-1, 1))
close_data = data['Close'].values.reshape(-1, 1)
# print(close_data[:5])

close_data = scaler.fit_transform(close_data)


def crete_dataset(data, time_step):
    data_x, data_y = [], []
    for i in range(len(data) - time_step-1):
        data_x.append(data[i:(i + time_step), 0])
        data_y.append(data[i + time_step, 0])
    return np.array(data_x), np.array(data_y)

time_step = 50
x_train, y_train = crete_dataset(close_data, time_step)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

print(x_train.shape)



Open 无缺失值
High 无缺失值
Low 无缺失值
Close 无缺失值
Volume 无缺失值
(1293, 50, 1)


In [66]:
## 构建LSTM模型
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))





ModuleNotFoundError: No module named 'tensorflow'